# 第二次作业

刘衍涛 计算机学院 2022281013229071

## 第一题
已知词项文档矩阵 C

In [1]:
import numpy as np

C = np.array([
    [1,0,1,0,0,0],
    [0,1,0,0,0,0],
    [1,1,0,0,0,0],
    [1,0,0,1,1,0],
    [0,0,0,1,0,1],

]
)
from scipy.linalg import svd
u, s, vh = svd(C)
print('\nthe top k singular value is')
print(s)
print('\nthe first 3 column of u')
print(u[:,:3])
print('\nthe first 3 row of vh is')
print(vh[:3,])


the top k singular value is
[2.16250096 1.59438237 1.27529025 1.         0.39391525]

the first 3 column of u
[[ 0.44034748 -0.29617436 -0.56949758]
 [ 0.12934635 -0.33145069  0.5870217 ]
 [ 0.47553026 -0.51111524  0.36768998]
 [ 0.70302032  0.35057241 -0.15490588]
 [ 0.26267284  0.64674677  0.4145917 ]]

the first 3 row of vh is
[[ 0.74862305  0.2797116   0.2036288   0.44656311  0.32509596  0.12146715]
 [-0.28645399 -0.52845914 -0.18576119  0.6255207   0.21987976  0.40564094]
 [-0.2797116   0.74862305 -0.44656311  0.2036288  -0.12146715  0.32509596]]


## 第二题
1)	实现任意一种基于机器学习（包括深度学习）的命名实体识别方法，并在公开数据集上进行评估。推荐使用CoNLL-2003数据集

In [20]:
from datasets import load_dataset
conll2003 = load_dataset('conll2003')

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

ner = pipeline("ner", model=model, tokenizer=tokenizer)

Found cached dataset conll2003 (/data/lyt/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 712.07it/s]


In [24]:
tag2num = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
num2tag = {v:k for k,v in tag2num.items()}

print(num2tag)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}


In [25]:
test_sents = [' '.join(unit['tokens']) for unit in conll2003['test']]
true_labels = [
    [{
        'label':num2tag[unit['ner_tags'][idx]],
        'start':len(' '.join(unit['tokens'][idx]))-len(unit['tokens'][idx]),
        'end':len(' '.join(unit['tokens'][idx]))
    } for idx in range(len(unit['ner_tags'])) if unit['ner_tags'][idx]!=0] for unit in conll2003['test'] 
]

In [21]:
predict_labels = ner(test_sents)

In [29]:
for unit in predict_labels:
    for label in unit:
        label['label'] = label['entity']

In [30]:
from nervaluate import Evaluator

evaluator = Evaluator(true_labels, predict_labels, tags=list(tag2num.keys()))

# Returns overall metrics and metrics for each tag

results, results_per_tag = evaluator.evaluate()

print(results)

{'ent_type': {'correct': 2177, 'incorrect': 1698, 'partial': 0, 'missed': 4023, 'spurious': 11581, 'possible': 7898, 'actual': 15456, 'precision': 0.1408514492753623, 'recall': 0.27563940238034945, 'f1': 0.18643487197054037}, 'partial': {'correct': 86, 'incorrect': 0, 'partial': 3789, 'missed': 4023, 'spurious': 11581, 'possible': 7898, 'actual': 15456, 'precision': 0.12813793995859213, 'recall': 0.2507596859964548, 'f1': 0.16960691958551}, 'strict': {'correct': 80, 'incorrect': 3795, 'partial': 0, 'missed': 4023, 'spurious': 11581, 'possible': 7898, 'actual': 15456, 'precision': 0.005175983436853002, 'recall': 0.010129146619397315, 'f1': 0.006851074762353343}, 'exact': {'correct': 86, 'incorrect': 3789, 'partial': 0, 'missed': 4023, 'spurious': 11581, 'possible': 7898, 'actual': 15456, 'precision': 0.005564182194616977, 'recall': 0.010888832615852115, 'f1': 0.007364905369529845}}


In [32]:
print(results_per_tag)

{'O': {'ent_type': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}, 'partial': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}, 'strict': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}, 'exact': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}}, 'B-PER': {'ent_type': {'correct': 567, 'incorrect': 180, 'partial': 0, 'missed': 804, 'spurious': 1673, 'possible': 1551, 'actual': 2420, 'precision': 0.23429752066115703, 'recall': 0.3655705996131528, 'f1': 0.28557038529337697}, 'partial': {'correct': 28, 'incorrect': 0, 'partial': 719, 'missed': 804, 'spurious': 1673, 'possible': 1551, 'actual': 2420, 'precision': 0.16012396694214875, 